# Solver that can play Semantle online

Online guessing with USE semantics and webdriver integration

In [ ]:
import random
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
import time

In [ ]:
import sys
sys.path.insert(0, '../src')

from cohort_bayes_solver import CohortBayesSolver
from gradient_solver import GradientSolver
import similarity_model as sm
import vocabulary as voc

## Similarity model

In [ ]:
base_vocabulary_uf = voc.VocabularyUF()
base_vocabulary_uf.remove_nsfw_words()

In [ ]:
similarity_use = sm.SimilarityModelUSE(base_vocabulary_uf)

In [ ]:
similarity_w2v = sm.SimilarityModelW2V(base_vocabulary_uf)

## Incorporate game history

In [ ]:
def merge_guesses(solver, guesses, scores):
    unmerged = []
    for i, gw in enumerate(guesses):
        merged = False
        if gw in solver.similarity.vocab_words:
            gi = solver.similarity.word_index(gw)
            if not solver.guess_merged(gi):
                solver.merge_guess(gi, scores[i], 100)
                merged = True
        if not merged:
            unmerged.append(i)
    return unmerged

## Read game and guess

In [ ]:
def live_guess(driver, guess):
    guess_input = driver.find_element(By.ID, "guess")
    guess_input.send_keys(guess)
    guess_input.send_keys(Keys.RETURN)

def guess_missed(driver):
    guess_error = driver.find_element(By.ID, "error")
    return guess_error.is_displayed()
    
def clear_guess(driver):
    guess_input = driver.find_element(By.ID, "guess")
    guess_input.clear()

In [ ]:
# todo - sometimes guesses are mapped to spelling variants
# (cancelled -> canceled) and not recorded as guesses

def get_scores(driver):
    guess_table = driver.find_element(By.ID, "guesses")
    rows = guess_table.find_elements(By.TAG_NAME, "tr")
    guesses = []
    scores = []
    for row in rows:
        cols = row.find_elements(By.TAG_NAME, "td")
        if len(cols) >= 3:
            guesses.append(cols[1].text)
            scores.append(float(cols[2].text))
    return guesses, scores

## Init solver and game

In [ ]:
#solver = CohortBayesSolver(similarity_w2v, precision=0.1, recall=1, log=True)
solver = GradientSolver(similarity_use, log=True)

In [ ]:
driver = webdriver.Firefox()
driver.get("https://semantle.com")
#driver.get("https://semantle.com/yesterday")
#driver.get("https://www.thewordfinder.com/semantle-archives/?puzzle=271")
assert "Semantle" in driver.title
try:
    elem = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.ID, "rules-close")))
finally:
    driver.find_element(By.ID, "rules-close").click()

## Automatically solve

In [ ]:
guesses, scores = [], []
while not (len(scores) > 0 and max(scores) > 99.9):
    guess = solver.similarity.word_string(solver.make_guess())
    print(guess)
    live_guess(driver, guess)
    time.sleep(1)
    missed = guess_missed(driver)
    if missed: solver.merge_guess(solver.similarity.word_index(guess), 0, 100)
    clear_guess(driver)
    if missed: time.sleep(1)
    try:
        guesses, scores = get_scores(driver)
    except StaleElementReferenceException:
        pass
    finally:
        merge_guesses(solver, guesses, scores)

In [ ]:
driver.quit()